In [ ]:
!nvidia-smi

Thu Aug 20 13:36:59 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/ShivamShrirao/dnn_from_scratch.git

Cloning into 'dnn_from_scratch'...
remote: Enumerating objects: 222, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 1440 (delta 143), reused 153 (delta 86), pack-reused 1218
Receiving objects: 100% (1440/1440), 13.74 MiB | 30.86 MiB/s, done.
Resolving deltas: 100% (958/958), done.


In [ ]:
!git clone https://github.com/ShivamShrirao/deep_Q_learning_from_scratch.git

Cloning into 'deep_Q_learning_from_scratch'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (220/220), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 220 (delta 135), reused 152 (delta 67), pack-reused 0
Receiving objects: 100% (220/220), 175.87 KiB | 3.52 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
%cd deep_Q_learning_from_scratch

/content/deep_Q_learning_from_scratch


In [ ]:
!git pull origin master

From https://github.com/ShivamShrirao/deep_Q_learning_from_scratch
 * branch            master     -> FETCH_HEAD
Already up to date.


In [3]:
import gym
import time
import cv2
import numpy as np
from collections import deque

from settings import *
from agent import *
from experience import *

In [4]:
agt = Agent(actions=[0,2,3], epsilon=1, min_epsilon=0.1, eps_decay=2e-6, target_update_thresh=1000)
D_exp = ReplayMemory(capacity=1_000_000, nlap=2)

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
- InputLayer(InputLayer)  (None, 80, 80, 4)          echo             0
__________________________________________________________________________________________
0 Conv2D(Conv2D)          (None, 40, 40, 32)         relu             1184
__________________________________________________________________________________________
1 Conv2D(Conv2D)          (None, 20, 20, 64)         relu             18496
__________________________________________________________________________________________
2 Conv2D(Conv2D)          (None, 10, 10, 128)        relu             73856
__________________________________________________________________________________________
3 Flatten(Flatten)        (None, 12800)              echo             0
__________________________________________________________________________________________
4 Den

In [5]:
D_exp.current_state.nbytes/1024/1024/1024

5.9604644775390625

In [6]:
env = gym.make('Pong-v0')

In [7]:
D_exp.len

0

In [14]:
agt.target_update_thresh = 1000

In [ ]:
for i_episode in range(500):
    obinit = env.reset()
    if not i_episode:
        observation = obinit
        state = preproc_obsv(observation)
        state_que = deque([], maxlen=NFRAMES)
        for i in range(NFRAMES):
            state_que.append(state)
    ep_score = 0
    start = time.time()
    for t in range(10_000):
        s_s = time.time()
#         env.render()
        state = preproc_obsv(observation)
        state_que.append(state)
        action = agt.get_action(state_que)
        next_observation, reward, done, info = env.step(action)
        ep_score += reward

        D_exp.store_transition(state, agt.actions.index(action), reward, done)
        observation = next_observation

        if (D_exp.len-D_exp.idx_len) > BATCH_SIZE:
            grads = agt.train(D_exp, BATCH_SIZE)
        if done:
            break
        print('\r', t, action, ep_score, end='  ')
    print(f"\rEpisode {i_episode+1} finished after {t+1} timesteps, Score: {ep_score}, Epsilon: {agt.epsilon:.6f}, Time: {time.time()-start:.2f}")
    if not i_episode%8:
        agt.model.save_weights("/content/drive/My Drive/model.w8s")
env.close()

Episode 1 finished after 2339 timesteps, Score: -19.0, Epsilon: 0.695322, Time: 30.79
Episode 2 finished after 2594 timesteps, Score: -18.0, Epsilon: 0.690134, Time: 33.87
Episode 3 finished after 1997 timesteps, Score: -17.0, Epsilon: 0.686140, Time: 26.49
Episode 4 finished after 2691 timesteps, Score: -18.0, Epsilon: 0.680758, Time: 35.36
Episode 5 finished after 2687 timesteps, Score: -18.0, Epsilon: 0.675384, Time: 35.25
Episode 6 finished after 2047 timesteps, Score: -21.0, Epsilon: 0.671290, Time: 26.78
Episode 7 finished after 1354 timesteps, Score: -21.0, Epsilon: 0.668582, Time: 17.78
Episode 8 finished after 2338 timesteps, Score: -20.0, Epsilon: 0.663906, Time: 30.66
Episode 9 finished after 1738 timesteps, Score: -19.0, Epsilon: 0.660430, Time: 22.68
Episode 10 finished after 2594 timesteps, Score: -20.0, Epsilon: 0.655242, Time: 34.12
Episode 11 finished after 2454 timesteps, Score: -18.0, Epsilon: 0.650334, Time: 32.37
Episode 12 finished after 2057 timesteps, Score: -20

In [16]:
agt.epsilon = 0.7

In [13]:
for i_episode in range(2):
    obinit = env.reset()
    if not i_episode:
        observation = obinit
        state = preproc_obsv(observation)
        state_que = deque([], maxlen=NFRAMES)
        for i in range(NFRAMES):
            state_que.append(state)
    ep_score = 0
    preds = []
    reward_history = []
    start = time.time()
    t = -1
    while 1:
        t+=1
        # env.render()
        state = preproc_obsv(observation)
        state_que.append(state)
        # action = agt.get_action(state_que)
        out = agt.predict(state_que)
        pidx = cp.argmax(out[0]).item()
        preds.append(out[0][pidx].item())
        action = agt.actions[pidx]
        next_observation, reward, done, info = env.step(action)
        ep_score += reward
        reward_history.append(reward)
        observation = next_observation
        # time.sleep(1/fps)
        if done:
            break
        print('\r', t, action, ep_score, end='  ')
    print(f"\rEpisode {i_episode+1} finished after {t+1} timesteps, Score: {ep_score}, Epsilon: {agt.epsilon:.6f}, Time: {time.time()-start:.2f}")

Episode 1 finished after 3200 timesteps, Score: -12.0, Epsilon: 0.000000, Time: 10.47
Episode 2 finished after 2793 timesteps, Score: -17.0, Epsilon: 0.000000, Time: 9.17
